In [ ]:
%pip install pwhl_api --upgrade
import pandas as pd
from pwhl_api.static.players import get_players, get_player, get_player_by_name
from pwhl_api.endpoints.playerskaterstats import PlayerSkaterStats
from pwhl_api.endpoints.playercareerstats import PlayerCareerStats

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Access Player Registry Data
The players module provides access to static player information including names, hometowns, and birthdates.

In [23]:
# Get all players in the registry
players = get_players()
print(f"Total players in registry: {len(players)}")
print("\nFirst 5 players:")
print(pd.DataFrame(players[:5]))

Total players in registry: 301

First 5 players:
  player_id                name         hometown   birthdate
0        49        Lexie Adzija   St. Thomas, ON  2000-06-30
1        18      Sophie Shirley    Saskatoon, SK  1999-06-30
2        15        Alina Müller  Winterthur, SWI  1998-03-12
3       111  Taylor Wenczkowski                   1997-09-25
4       176    Caitrin Lonergan                   1997-09-10


In [24]:
# Search for a specific player by name
player = get_player_by_name("Brianne Jenner")
if player:
    print(f"Found player: {player['name']}")
    print(f"  Player ID: {player['player_id']}")
    print(f"  Hometown: {player['hometown']}")
    print(f"  Birthdate: {player['birthdate']}")
else:
    print("Player not found")

Found player: Brianne Jenner
  Player ID: 58
  Hometown: Oakville, ON
  Birthdate: 1991-05-04


In [25]:
# Search for a player by ID
player = get_player("58")
if player:
    print(f"Found player: {player['name']}")
    print(f"  Hometown: {player['hometown']}")
else:
    print("Player not found")

Found player: Brianne Jenner
  Hometown: Oakville, ON


## 2. Analyze Current Season Skater Statistics
Get statistics for all skaters in the current season and analyze top performers.

In [26]:
# Get all forwards for season 8
forwards = PlayerSkaterStats(season="8", position="forwards")
forwards_df = forwards.get_data_frame()

print(f"Total forwards: {len(forwards_df)}")
print(f"\nColumns available: {list(forwards_df.columns)}")

Total forwards: 109

Columns available: ['player_id', 'name', 'active', 'position', 'rookie', 'team_code', 'games_played', 'goals', 'shots', 'hits', 'shots_blocked_by_player', 'ice_time_minutes_seconds', 'shooting_percentage', 'assists', 'points', 'points_per_game', 'plus_minus', 'penalty_minutes', 'penalty_minutes_per_game', 'ice_time_per_game_avg', 'hits_per_game_avg', 'power_play_goals', 'power_play_assists', 'short_handed_goals', 'short_handed_assists', 'faceoff_attempts', 'faceoff_wins', 'faceoff_pct', 'rank']


In [27]:
# Convert numeric columns and find top scorers
forwards_df['points'] = pd.to_numeric(forwards_df['points'], errors='coerce')
forwards_df['goals'] = pd.to_numeric(forwards_df['goals'], errors='coerce')
forwards_df['assists'] = pd.to_numeric(forwards_df['assists'], errors='coerce')
forwards_df['games_played'] = pd.to_numeric(forwards_df['games_played'], errors='coerce')

# Top 10 point scorers
top_scorers = forwards_df.nlargest(10, 'points')[['name', 'team_code', 'games_played', 'goals', 'assists', 'points']]
print("Top 10 Point Scorers (Forwards):")
print(top_scorers)

Top 10 Point Scorers (Forwards):
                      name team_code  games_played  goals  assists  points
0  Kendall Coyne Schofield       MIN            10      8        3      11
1      Marie-Philip Poulin       MTL             9      5        6      11
2      Britta Curl-Salemme       MIN            10      4        7      11
3           Brianne Jenner       OTT            10      5        5      10
4               Abby Roque       MTL             9      4        6      10
5             Kelly Pannek       MIN            10      4        6      10
6              Daryl Watts       TOR             9      5        3       8
7            Julia Gosling       SEA             8      4        4       8
8           Rebecca Leslie       OTT            10      4        4       8
9           Alex Carpenter       SEA             8      5        2       7


In [28]:
# Calculate and find leaders in points per game
forwards_df['ppg'] = forwards_df['points'] / forwards_df['games_played']

# Filter for players with at least 5 games
ppg_leaders = forwards_df[forwards_df['games_played'] >= 5].nlargest(10, 'ppg')[['name', 'team_code', 'games_played', 'ppg']]
print("\nPoints Per Game Leaders (min 5 games):")
print(ppg_leaders)


Points Per Game Leaders (min 5 games):
                       name team_code  games_played       ppg
1       Marie-Philip Poulin       MTL             9  1.222222
4                Abby Roque       MTL             9  1.111111
0   Kendall Coyne Schofield       MIN            10  1.100000
2       Britta Curl-Salemme       MIN            10  1.100000
3            Brianne Jenner       OTT            10  1.000000
5              Kelly Pannek       MIN            10  1.000000
7             Julia Gosling       SEA             8  1.000000
6               Daryl Watts       TOR             9  0.888889
9            Alex Carpenter       SEA             8  0.875000
11            Hilary Knight       SEA             8  0.875000


In [29]:
# Get defenders statistics
defenders = PlayerSkaterStats(season="8", position="defenders")
defenders_df = defenders.get_data_frame()

defenders_df['points'] = pd.to_numeric(defenders_df['points'], errors='coerce')
defenders_df['games_played'] = pd.to_numeric(defenders_df['games_played'], errors='coerce')

print(f"\nTotal defensemen: {len(defenders_df)}")
print("\nTop 5 Defensemen by Points:")
print(defenders_df.nlargest(5, 'points')[['name', 'team_code', 'games_played', 'points']])


Total defensemen: 184

Top 5 Defensemen by Points:
                      name team_code  games_played  points
0  Kendall Coyne Schofield       MIN            10      11
1      Marie-Philip Poulin       MTL             9      11
2      Britta Curl-Salemme       MIN            10      11
3           Brianne Jenner       OTT            10      10
4               Abby Roque       MTL             9      10


## 3. Analyze Player Career Statistics
Get detailed career and season-by-season statistics for a specific player.

In [30]:
# Get career stats for a player
player_stats = PlayerCareerStats(player_id="58", season_id="8", statsType="standard")
dfs = player_stats.get_data_frames()

print("Player Career Data Frames Available:")
print(f"  - playerStats (regularSeason, playoffs)")
print(f"  - gameByGame")
print(f"  - currentSeasonStats")
print(f"  - playerShots")

Player Career Data Frames Available:
  - playerStats (regularSeason, playoffs)
  - gameByGame
  - currentSeasonStats
  - playerShots


In [31]:
# View current season statistics
current_season = dfs['currentSeasonStats']
print("Current Season Statistics:")
print(current_season)

# Extract key stats
if not current_season.empty:
    games_played = current_season['games_played'].values[0]
    points = current_season['points'].values[0]
    print(f"\nGames Played: {games_played}")
    print(f"Points: {points}")

Current Season Statistics:
              season_name games_played goals assists points penalty_minutes
0  2025-26 Regular Season           10     5       5     10               2

Games Played: 10
Points: 10


In [32]:
# Analyze game-by-game performance
game_by_game = dfs['gameByGame']

# Convert to numeric
game_by_game['points'] = pd.to_numeric(game_by_game['points'], errors='coerce')

# Find games with multiple points
multi_point_games = game_by_game[game_by_game['points'] >= 2]

print(f"\nGames with 2+ points: {len(multi_point_games)} out of {len(game_by_game)}")
print(f"Highest single game points: {game_by_game['points'].max()}")
print(f"\nRecent games with 2+ points:")
print(multi_point_games[['date_played', 'game_id','game', 'goals', 'assists', 'points']].head(5))


Games with 2+ points: 3 out of 10
Highest single game points: 4

Recent games with 2+ points:
  date_played game_id       game goals assists  points
1  2025-11-26     215  VAN @ OTT     2       2       4
7  2025-12-23     235  OTT @ TOR     0       2       2
9  2026-01-03     244  MIN @ OTT     2       0       2


In [33]:
# View career regular season statistics
career_regular = dfs['playerStats']['regularSeason']
print("Career Regular Season Statistics:")
print(career_regular[['season_name', 'games_played', 'goals', 'assists', 'points']].head(10))

Career Regular Season Statistics:
              season_name games_played goals assists points
0  2025-26 Regular Season           10     5       5     10
1  2024-25 Regular Season           28     7       8     15
2     2024 Regular Season           24     9      11     20
3                   Total           62    21      24     45


## 4. Compare Multiple Players
Compare statistics across multiple players.

In [34]:
# Get current season stats for multiple players
player_ids = ["58", "57", "72", "220"]  # Example player IDs
player_comparison = []

for pid in player_ids:
    try:
        stats = PlayerCareerStats(player_id=pid, season_id="8", statsType="standard")
        dfs = stats.get_data_frames()
        current = dfs['currentSeasonStats']
        
        if not current.empty:
            # Get player name from registry
            player = get_player(pid)
            player_name = player['name'] if player else f"Player {pid}"
            
            player_comparison.append({
                'player_id': pid,
                'name': player_name,
                'games_played': current['games_played'].values[0],
                'goals': current['goals'].values[0],
                'assists': current['assists'].values[0],
                'points': current['points'].values[0]
            })
    except Exception as e:
        print(f"Error getting stats for player {pid}: {e}")

comparison_df = pd.DataFrame(player_comparison)
print("\nPlayer Comparison (Current Season):")
print(comparison_df.sort_values('points', ascending=False))


Player Comparison (Current Season):
  player_id            name games_played goals assists points
2        72  Rebecca Leslie           10     4       4      8
1        57   Gabbie Hughes            8     1       5      6
3       220  Mannon McMahon           10     1       1      2
0        58  Brianne Jenner           10     5       5     10
